In [1]:
import math

import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm.auto import tqdm
from scipy.stats import norm
import matplotlib as mpl
mpl.use("TkAgg")
import ddpm
import datasets

In [2]:
sizes = range(100, 5000, 400)
sizes = [2500]
names = [f"point_2d{s}" for s in sizes]

In [3]:
for i in range(len(sizes)):
    print(sizes[i])
    !python ddpm.py --dataset circle --experiment_name {names[i]} --num_epochs 50 --dataset_size {sizes[i]} --beta_schedule ours --num_timesteps 1000

2500
sMIN GAMMA tensor(4.3165e-06, dtype=torch.float64) LEN 50
Training model 0...
END LOSS WAS 1.0378519071325878
Saving model...
Training model 1...
END LOSS WAS 0.8919519670135705
Saving model...
Training model 2...
END LOSS WAS 0.9048583430954108
Saving model...
Training model 3...
END LOSS WAS 0.9335633894609214
Saving model...
Training model 4...
END LOSS WAS 0.7973158165292966
Saving model...
Training model 5...
END LOSS WAS 0.7869084193318385
Saving model...
Training model 6...
END LOSS WAS 0.827351789152277
Saving model...
Training model 7...
END LOSS WAS 0.03150011332932531
Saving model...
Training model 8...
END LOSS WAS 0.01953048085248614
Saving model...
Training model 9...
END LOSS WAS 0.00736420549120336
Saving model...
Training model 10...
END LOSS WAS 0.00913519150172068
Saving model...
Training model 11...
END LOSS WAS 0.004931943196921421
Saving model...
Training model 12...
END LOSS WAS 0.0013446011370140468
Saving model...
Training model 13...
END LOSS WAS 8.941738

In [19]:
def calculate_stats(model_dir, dataset='point', score='model'):
    num_scheduler_timesteps = 1000
    noise_scheduler = ddpm.NoiseScheduler(num_timesteps=num_scheduler_timesteps, beta_schedule='ours')
    num_timesteps = len(noise_scheduler)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    models = [ddpm.MLP(input_dim=2) for _ in range(num_timesteps)]
    for t in range(num_timesteps):
        path = model_dir + f"/model{t}.pth"
        models[t].load_state_dict(torch.load(path))
        models[t].to(device)
        models[t].eval()
    eval_batch_size = 100000
    plot_step = 1
    num_timesteps = len(noise_scheduler.betas)
    curr_vars = torch.sqrt(1 - torch.exp(-2 * noise_scheduler.times))
    sample = torch.randn(eval_batch_size, 2).to(device).to(device)
    timesteps = list(range(num_timesteps))[::-1][:-5]
    print("LAST STD", curr_vars[timesteps[-1]].item())
    samples = []
    steps = []
    for i, t in enumerate(tqdm(timesteps)):
        t_int = t
        t = torch.from_numpy(np.repeat(t, eval_batch_size)).long().to(device)
        with torch.no_grad():
            variance = torch.sqrt(1 - noise_scheduler.alphas_cumprod[t])
            v = curr_vars[t].cpu().numpy()
            if score == 'model':
                residual = models[t_int](sample, t)
            else:
                residual = sample / variance.unsqueeze(-1)
        sample = noise_scheduler.step(residual, t[0], sample)
        if (i + 1) % plot_step == 0:
            sample_cpu = sample.cpu()
            samples.append(sample_cpu.numpy())
            steps.append(i + 1)
    if dataset == 'ret':
        return samples
    elif dataset == 'square':
        return process_square(samples[-1], mode='median')
    elif dataset == 'point2d':
        return process_point2d(samples[-1], mode='median')
    elif dataset == 'circle':
        return process_circle(samples[-1], r=3, mode='median')
    elif dataset == 'point':
        return process_point(samples[-1], mode='median')
    else:
        raise ValueError("INVALID DATASET")
def process_square(samples, r=3, mode='median'):
    t = 0
    d = []
    for s in samples:
        dists = [abs(s[0] - r), abs(s[0] + r), abs(s[1] - r), abs(s[1] + r)]
        m = min(dists)
        d.append(m)
    # d = sorted(d)[:-round(len(d)*0.02)]
    if mode == 'median':
        return sorted(d)[len(d)//2]
    for m in d:
        t += m**2
    t /= len(d)
    return np.sqrt(t)
def process_point2d(samples, mode='median'):
    norms = np.apply_along_axis(np.linalg.norm, 1, samples)
    print("MEAN", np.mean(norms), "MEDIAN", np.median(norms))
    return np.median(norms)
    # med = np.median(norms)
def process_point(samples, mode='median'):
    return np.median(samples)
    centered = samples.squeeze() - np.mean(samples.squeeze())
    print(np.mean(samples.squeeze()))
    return np.median(centered)

In [20]:
s = []
for i in range(len(sizes)):
    print(sizes[i], names[i])
    s.append(np.abs(calculate_stats(f"exps/{names[i]}", dataset='point2d', score='model')))
plt.clf()
plt.scatter(sizes, np.array(s))
plt.yscale('log')
plt.title("median error vs dataset size")
plt.ylabel("median error")
plt.xlabel("dataset size")
plt.show()

2500 point_2d2500
sMIN GAMMA tensor(4.3165e-06, dtype=torch.float64) LEN 50
LAST STD 4.220806844430688e-05


  0%|          | 0/45 [00:00<?, ?it/s]

MEAN 0.0020550592167652673 MEDIAN 6.0824007237356736e-05


In [103]:
s2 = [1 / x for x in s]
plt.clf()
plt.scatter(sizes, np.array(s2))
# plt.yscale('log')
plt.title("median error vs dataset size")
plt.ylabel("median error")
plt.xlabel("dataset size")
plt.savefig(f'static/devs_point2d.png', bbox_inches='tight')
plt.show()

In [7]:
model_dir = f"exps/{names[-1]}"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
noise_scheduler = ddpm.NoiseScheduler(num_timesteps=1000, beta_schedule='ours')
num_timesteps = len(noise_scheduler.times)
models = [ddpm.MLP(input_dim=2) for _ in range(num_timesteps)]
for t in range(num_timesteps):
    path = model_dir + f"/model{t}.pth"
    models[t].load_state_dict(torch.load(path))
    models[t].to(device)
    models[t].eval()

sMIN GAMMA tensor(4.3165e-06, dtype=torch.float64) LEN 50


In [8]:
curr_stds = torch.sqrt(1 - torch.exp(-2 * noise_scheduler.times))
print(curr_stds)
t = 10
v = curr_stds[t].item()
print(v)
x_scale = np.linspace(-v * 5, v * 5, 1000)
# x_scale = np.linspace(-10, 10, 1000)
inputs = torch.tensor(x_scale, device=device).unsqueeze(1)
times = torch.ones(len(inputs)).to(device) * t
model_residuals = models[t](inputs, times)
true_residuals = inputs / v

tensor([4.3165e-06, 7.7347e-06, 1.2284e-05, 1.8768e-05, 2.8238e-05, 4.2208e-05,
        6.2907e-05, 9.3635e-05, 1.3929e-04, 2.0716e-04, 3.0805e-04, 4.5806e-04,
        6.8110e-04, 1.0127e-03, 1.5058e-03, 2.2390e-03, 3.3291e-03, 4.9500e-03,
        7.3599e-03, 1.0943e-02, 1.6269e-02, 2.4184e-02, 3.5938e-02, 5.3369e-02,
        7.9136e-02, 1.1696e-01, 1.7168e-01, 2.4849e-01, 3.5035e-01, 4.7334e-01,
        6.0371e-01, 7.2296e-01, 8.1800e-01, 8.8584e-01, 9.3060e-01, 9.5865e-01,
        9.7566e-01, 9.8578e-01, 9.9173e-01, 9.9520e-01, 9.9722e-01, 9.9839e-01,
        9.9907e-01, 9.9946e-01, 9.9969e-01, 9.9982e-01, 9.9990e-01, 9.9994e-01,
        9.9997e-01, 9.9998e-01], device='cuda:0', dtype=torch.float64)
0.0003080510901544058


IndexError: index 1 is out of bounds for dimension 1 with size 1

In [10]:
plt.plot(x_scale, model_residuals.data.cpu().numpy(), label='model')
plt.plot(x_scale, true_residuals.data.cpu().numpy(), label='true')
for y in (np.arange(11)-5)*v:
    plt.axvline(y, alpha=1 if y == 0 else 0.2)
plt.legend()
plt.savefig("score.png")
plt.show()

In [14]:
errors = []
for t in range(50):
    print(t)
    v = curr_vars[t].cpu().numpy()
    x_range = np.linspace(-v*5, v*5, 1000)
    diff = x_range[1] - x_range[0]
    l2 = 0
    difc = 0
    pc = 0
    tot = 0
    for i in x_range:
        v = curr_vars[t]
        pdf = norm.pdf(i, 0, v.item())
        model_val = model(torch.tensor([[i]], device=device, dtype=torch.float32), torch.ones(1, device=device, dtype=torch.float32)*t)
        true_val = torch.tensor([[i]], device=device) / torch.sqrt(1 - torch.exp(-2 * noise_scheduler.times[t]))
        error = (model_val.data.cpu().numpy() - true_val.data.cpu().numpy())[0]
        l2 += (error**2)*diff*pdf
        difc += diff
        pc += pdf
        tot += diff*pdf
    print(l2, difc, tot, v.item()*2)
    errors.append(l2*v.item()*v.item())
    # break

0


NameError: name 'curr_vars' is not defined

In [96]:
x_range = np.linspace(-10, 10, 1000)
diff = x_range[1] - x_range[0]
tot = 0
for i in x_range:
    tot += diff * norm.pdf(i, 0, 10)
print(tot)

0.6831737563750486


In [12]:
t = 5
slice_index = 1
print(names[-1])
model_path = f"exps/{names[-1]}/model{t}.pth"
model = ddpm.MLP(input_dim=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()
noise_scheduler = ddpm.NoiseScheduler(num_timesteps=1000, beta_schedule="ours")
curr_vars = torch.sqrt(1 - torch.exp(-2 * noise_scheduler.times))
v = curr_vars[t].item()
scores = []
x_scale = torch.linspace(-v * 5, v * 5, 1000)
if slice_index == 0:
    inputs = torch.stack((x_scale, torch.zeros(1000)), dim=1).to(device)
else:
    inputs = torch.stack((torch.zeros(1000), x_scale), dim=1).to(device)
print(noise_scheduler.betas.shape)
times = torch.ones(len(inputs), device=device)*t
model_residuals = model(inputs, times)
true_residuals = inputs / v

point_2d4900
sMIN GAMMA tensor(4.3165e-06, dtype=torch.float64) LEN 50
torch.Size([50])


In [13]:
plt.clf()
plt.plot(x_scale, model_residuals.data.cpu().numpy()[:, slice_index], label='model')
plt.plot(x_scale, true_residuals.data.cpu().numpy()[:, slice_index], label='true')
for y in (np.arange(11)-5)*v:
    plt.axvline(y, alpha=1 if y == 0 else 0.2)
plt.legend()
plt.savefig("score.png")
plt.show()
plt.clf()

In [96]:
a = np.array([[1, 4], [1, 1], [3, 4]])
np.median(np.apply_along_axis(np.linalg.norm, 1, a))

4.123105625617661